Import nessesary packages

In [1]:
from googleapiclient.discovery import build
import csv
import pandas as pd
import langid

API Key for use of the Youtube Data v3 API

In [2]:
API_KEY = ''

Function that gather youtube comments using YouTube v3 API
- equests comments from video
- checks for more pages
- max of 50000 comments per video

In [3]:
#gathering comments
def get_youtube_comments(video_id, max_results=100, max_comments=50000):
    video_comments = []
    comment_count = 0
    youtube = build("youtube", "v3", developerKey=API_KEY)

    # try a request
    try:
        video_response = youtube.commentThreads().list(part="snippet", videoId=video_id, maxResults=max_results).execute()
    except Exception as e:
        return video_comments

    # get comments
    while video_response:
        for item in video_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            video_comments.append(comment)
            comment_count += 1

            # check if the limit was reached
            if comment_count >= max_comments:
                return video_comments

        # check for more pages
        nextPageToken = video_response.get('nextPageToken')
        if nextPageToken:
            try:
                video_response = youtube.commentThreads().list(part="snippet", videoId=video_id, maxResults=max_results, pageToken=nextPageToken).execute()
            except Exception as e:
                break
        else:
            break
    return video_comments

takes comments and puts them into a csv fle

In [7]:
def to_CSV(name, comments):
    try:
        #open file or create new one 
        with open(name, mode='a', newline='') as file:
            writer = csv.writer(file)

            #if no header, make header
            if file.tell() == 0:
                writer.writerow(['Index', 'Comments'])

            #write comments to csv
            for index, value in enumerate(comments):
                writer.writerow([index, value])

        print(f"Data successfully written to {name}")

    except Exception as e:
        print(f"Error writing to CSV file: {e}")

cleans a given csv file
- uses langid to fid only english comments

In [4]:
def clean_csv(input_file, output_file):
    df = pd.read_csv(input_file)

    #check if comment english
    def is_english(comment):
        try:
            lang, confidence = langid.classify(comment)
            return lang == 'en' and confidence > 0.5
        except Exception:
            return False

    #check comments
    df['IsEnglish'] = df['Comments'].apply(is_english)
    df = df[df['IsEnglish'] & df['Index'].notnull()]

    #save engish comment
    df[['Index', 'Comments']].to_csv(output_file, index=False)

Get top 5 videos by category and gather comments

In [5]:
top_gaming = ['hI1MMVt7xEo', 'urHuO7Zbhhw', 'fPPGz5Qxw8A', 'BJPc49z57bU']
top_vlogs = ['HhM0BYCHL00', 'briN_W5yzYY','zU95-wX0xJo', '84WIaK3bl_s', 'WxfZkMm3wcg']
top_music = ['JGwWNGJdvx8', 'RgKAFK5djSk', 'OPf0YbXqDm0', '9bZkp7q19f0', '09R8_2nJtjg']
top_beauty = ['ex33wtqnNz8', 'OO3NO29L50U', 'mGs4CjeJiJQ', 'VD47yv2NfMw', 'Fz-DTp2iewQ']
top_reaction = ['-XKdCBZEWeU', 'KFChuq6piZ8', 'dYh6R4Jhxoo', 'KyNJD8Ewcjk', 'm5m8NoPRkRw']
top_controversial = ['2pMsYSR7g78', 'FV8B9jYWlRQ', 'pJ9PKQbkJv8']

In [ ]:
for videoid in top_gaming:
    comments = get_youtube_comments(videoid)
    to_CSV('data/RawCSV/gaming_comments.csv',comments)

In [ ]:
for videoid in top_vlogs:
    comments = get_youtube_comments(videoid)
    to_CSV('data/RawCSV/vlog_comments.csv',comments)

In [ ]:
for videoid in top_music:
    comments = get_youtube_comments(videoid)
    to_CSV('data/RawCSV/music_comments.csv',comments)

In [ ]:
for videoid in top_beauty:
    comments = get_youtube_comments(videoid)
    to_CSV('data/RawCSV/beauty_comments.csv',comments)

In [ ]:
for videoid in top_reaction:
    comments = get_youtube_comments(videoid)
    to_CSV('data/RawCSV/reaction_comments.csv',comments)

In [ ]:
for videoid in top_controversial:
    comments = get_youtube_comments(videoid)
    to_CSV('controversial_comments.csv', comments)

In [4]:
clean_csv('data/RawCSV/gaming_comments.csv', 'data/CleanCSV/gaming_comments_clean.csv')
clean_csv('data/RawCSV/vlog_comments.csv', 'data/CleanCSV/vlog_comments_clean.csv')
clean_csv('data/RawCSV/music_comments.csv', 'data/CleanCSV/music_comments_clean.csv')
clean_csv('data/RawCSV/beauty_comments.csv', 'data/CleanCSV/beauty_comments_clean.csv')
clean_csv('data/RawCSV/reaction_comments.csv', 'data/CleanCSV/reaction_comments_clean.csv')
clean_csv('data/RawCSV/controversial_comments.csv', 'data/CleanCSV/controversial_comments_clean.csv')